<a href="https://colab.research.google.com/github/toplyn/structured/blob/master/Python_Exercise_4_Using_CensusReporter_to_Make_Geographic_Selections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code analyzes various parameters that may support a hypothetical coffee shop expansion to new markets.  The shop may see the best success with individuals 18-34, with incomes 50k-75k and with higher hispanic populations.  The code hits a census API to pull demographic data for different areas to find potential locations.

**Best Demographics for Organic coffee targeting**
- $50k - $75k income
- 18 - 34 years
- Targeting hispanic populations

**Question 1 - Highest income**
- 	New York-Newark-Jersey City, NY-NJ-PA	

**Question 2 - Highest in age range**
- 	New York-Newark-Jersey City, NY-NJ-PA	

**Question 3 - Highest hispanic population**
- 	Los Angeles-Long Beach-Anaheim, CA

**Question 4 - Highest percentage across the 3**
- New York-Newark-Jersey City, NY-NJ-PA	


In [0]:
## Import libraries
import pandas as pd
import json
import requests
import time
import numpy as np
import seaborn as sns

In [0]:
## Load data from cloud location
ozo = pd.read_csv('https://s3.amazonaws.com/vargo.aprd6342/data/msas.csv')

In [0]:
## tables from censusreporter that highlight information needed
## includes ages, incomes and race information
tableids = ['B01001','B03002','B19001']

In [6]:
## Join tables to make one list
tableidstring = ','.join(tableids)
print(tableidstring)

B01001,B03002,B19001


In [0]:
## set URL to query censusreporter API
requesturl = 'https://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=310|01000US' % str(tableidstring)

In [0]:
# Make API request
loaded = requests.get(requesturl)

In [0]:
# Pull JSON
parsed = loaded.json()

In [0]:
## Identify each level of information from JSON and add to OZO dataset
ozo['m1819'] = 0
ozo['m20'] = 0
ozo['m21'] = 0
ozo['m2224'] = 0
ozo['m2529'] = 0
ozo['m3034'] = 0
ozo['f1819'] = 0
ozo['f20'] = 0
ozo['f21'] = 0
ozo['f2224'] = 0
ozo['f2529'] = 0
ozo['f3034'] = 0
ozo['hispanic'] = 0
ozo['inc5059'] = 0
ozo['inc6074'] = 0
for geoid in list(ozo['CBSA']):
  geo = '31000US' + str(geoid)
  ozo.loc[ozo.CBSA == geoid, ['m1819']] = parsed['data'][geo]['B01001']['estimate']['B01001007'] 
  ozo.loc[ozo.CBSA == geoid, ['m20']] = parsed['data'][geo]['B01001']['estimate']['B01001008']
  ozo.loc[ozo.CBSA == geoid, ['m21']] = parsed['data'][geo]['B01001']['estimate']['B01001009']
  ozo.loc[ozo.CBSA == geoid, ['m2224']] = parsed['data'][geo]['B01001']['estimate']['B01001010']
  ozo.loc[ozo.CBSA == geoid, ['m2529']] = parsed['data'][geo]['B01001']['estimate']['B01001011']
  ozo.loc[ozo.CBSA == geoid, ['m3034']] = parsed['data'][geo]['B01001']['estimate']['B01001021']
  ozo.loc[ozo.CBSA == geoid, ['f1819']] = parsed['data'][geo]['B01001']['estimate']['B01001031']
  ozo.loc[ozo.CBSA == geoid, ['f20']] = parsed['data'][geo]['B01001']['estimate']['B01001032']
  ozo.loc[ozo.CBSA == geoid, ['f21']] = parsed['data'][geo]['B01001']['estimate']['B01001033']
  ozo.loc[ozo.CBSA == geoid, ['f2224']] = parsed['data'][geo]['B01001']['estimate']['B01001034']
  ozo.loc[ozo.CBSA == geoid, ['f2529']] = parsed['data'][geo]['B01001']['estimate']['B01001035']
  ozo.loc[ozo.CBSA == geoid, ['f3034']] = parsed['data'][geo]['B01001']['estimate']['B01001036']
  ozo.loc[ozo.CBSA == geoid, ['hispanic']] = parsed['data'][geo]['B03002']['estimate']['B03002012']
  ozo.loc[ozo.CBSA == geoid, ['inc5059']] = parsed['data'][geo]['B19001']['estimate']['B19001011']
  ozo.loc[ozo.CBSA == geoid, ['inc6074']] = parsed['data'][geo]['B19001']['estimate']['B19001012']

In [0]:
# ## Age data
# data['data']['31000US19100']['B01001']['estimate']['B01001007'] # male 18-19
# data['data']['31000US19100']['B01001']['estimate']['B01001008'] # male 20
# data['data']['31000US19100']['B01001']['estimate']['B01001009'] # male 21
# data['data']['31000US19100']['B01001']['estimate']['B010010010'] # male 22-24
# data['data']['31000US19100']['B01001']['estimate']['B010010011'] # male 25-29
# data['data']['31000US19100']['B01001']['estimate']['B010010012'] # male 30-34
# data['data']['31000US19100']['B01001']['estimate']['B010010031'] # female 18-19
# data['data']['31000US19100']['B01001']['estimate']['B010010032'] # female 20
# data['data']['31000US19100']['B01001']['estimate']['B010010033'] # female 21
# data['data']['31000US19100']['B01001']['estimate']['B010010034'] # female 22-24
# data['data']['31000US19100']['B01001']['estimate']['B010010035'] # female 25-29
# data['data']['31000US19100']['B01001']['estimate']['B010010036'] # female 30-34

In [0]:
## hispanic population
# data['data']['31000US19100']['B03002']['estimate']['B03002012']

In [0]:
# ## Income in certain range
# data['data']['31000US19100']['B19001']['estimate']['B19001011'] # 50-59k
# data['data']['31000US19100']['B19001']['estimate']['B19001012'] # 60-74k


In [0]:
# data['tables']['B19001']['columns']

In [0]:
## Create new column with all age groups and income groups
ozo['total_pop'] = ozo['m1819'] + ozo['m20'] + ozo['m21'] + ozo['m2224'] + \
                   ozo['m2529'] + ozo['m3034'] + ozo['f1819'] + ozo['f20'] + \
                   ozo['f21'] + ozo['f2224'] + ozo['f2529'] + ozo['f3034'] 
ozo['total_income'] = ozo['inc5059'] + ozo['inc6074']

In [15]:
## Highest population in my range 18 - 34
ozo.loc[ozo.total_pop == ozo.total_pop.max()]

,CBSA,NAME,POPESTIMATE2017,m1819,m20,m21,m2224,m2529,m3034,f1819,f20,f21,f2224,f2529,f3034,hispanic,inc5059,inc6074,total_pop,total_income
249,35620,"New York-Newark-Jersey City, NY-NJ-PA",20320876,248434.0,133732.0,132580.0,411899.0,750663.0,242555.0,237913.0,129681.0,129093.0,416225.0,762402.0,730200.0,4863602.0,444453.0,606435.0,4325377.0,1050888.0


In [0]:
## Highest population in income range $50k - $74k
ozo.loc[ozo.total_income == ozo.total_income.max()]

,CBSA,NAME,POPESTIMATE2017,m1819,m20,m21,m2224,m2529,m3034,f1819,f20,f21,f2224,f2529,f3034,hispanic,inc5059,inc6074,total_pop,total_income
249,35620,"New York-Newark-Jersey City, NY-NJ-PA",20320876,248434.0,133732.0,132580.0,411899.0,750663.0,242555.0,237913.0,129681.0,129093.0,416225.0,762402.0,730200.0,4863602.0,444453.0,606435.0,4325377.0,1050888.0


In [0]:
## Highest population of hispanic people
ozo.loc[ozo.hispanic == ozo.hispanic.max()]

,CBSA,NAME,POPESTIMATE2017,m1819,m20,m21,m2224,m2529,m3034,f1819,f20,f21,f2224,f2529,f3034,hispanic,inc5059,inc6074,total_pop,total_income
210,31080,"Los Angeles-Long Beach-Anaheim, CA",13353907,180803.0,100836.0,101477.0,294903.0,544334.0,140940.0,176190.0,97896.0,94470.0,290607.0,519755.0,480658.0,5972751.0,300860.0,399821.0,3022869.0,700681.0


In [0]:
## Create percentages
ozo['pop_perc'] = 0.0
ozo['inc_perc'] = 0.0
ozo['hispanic_perc'] = 0.0
ozo['pop_perc'] = ozo['total_pop'] / ozo['total_pop'].max()
ozo['inc_perc'] = ozo['total_income'] / ozo['total_income'].max()
ozo['hispanic_perc'] = ozo['hispanic'] / ozo['hispanic'].max()

In [0]:
## Highest mean group
ozo['mean_perc'] = ozo[['pop_perc','inc_perc','hispanic_perc']].mean(axis=1)

In [0]:
ozo.loc[ozo.mean_perc == ozo.mean_perc.max()]

,CBSA,NAME,POPESTIMATE2017,m1819,m20,m21,m2224,m2529,m3034,f1819,f20,f21,f2224,f2529,f3034,hispanic,inc5059,inc6074,total_pop,total_income,pop_perc,inc_perc,hispanic_perc,mean_perc
249,35620,"New York-Newark-Jersey City, NY-NJ-PA",20320876,248434.0,133732.0,132580.0,411899.0,750663.0,242555.0,237913.0,129681.0,129093.0,416225.0,762402.0,730200.0,4863602.0,444453.0,606435.0,4325377.0,1050888.0,1.0,1.0,0.814298,0.938099


In [0]:
ozo.head()


,CBSA,NAME,POPESTIMATE2017,m1819,m20,m21,m2224,m2529,m3034,f1819,f20,f21,f2224,f2529,f3034,hispanic,inc5059,inc6074,total_pop,total_income,pop_perc,inc_perc,hispanic_perc,mean_perc
0,10180,"Abilene, TX",170219,2955.0,1635.0,1496.0,5711.0,7218.0,1938.0,2667.0,1545.0,1815.0,4561.0,5723.0,5271.0,38892.0,5409.0,6161.0,42535.0,11570.0,0.009834,0.011010,0.006512,0.009118
1,10420,"Akron, OH",703505,10724.0,5803.0,4554.0,16178.0,23309.0,9880.0,10876.0,5953.0,5615.0,15540.0,22650.0,20902.0,13318.0,23051.0,29395.0,151984.0,52446.0,0.035138,0.049906,0.002230,0.029091
2,10500,"Albany, GA",151434,2270.0,1614.0,832.0,2939.0,5062.0,2077.0,2416.0,1183.0,1440.0,3165.0,5390.0,4996.0,4057.0,4471.0,5058.0,33384.0,9529.0,0.007718,0.009068,0.000679,0.005822
3,10540,"Albany, OR",125047,1279.0,977.0,606.0,2145.0,4010.0,2530.0,1208.0,984.0,634.0,1915.0,4028.0,3837.0,10483.0,3859.0,5414.0,24153.0,9273.0,0.005584,0.008824,0.001755,0.005388
4,10580,"Albany-Schenectady-Troy, NY",886188,15282.0,7901.0,7458.0,20534.0,28615.0,12075.0,14521.0,7365.0,7040.0,19708.0,27717.0,27902.0,42999.0,26429.0,35665.0,196118.0,62094.0,0.045341,0.059087,0.007199,0.037209
